<a href="https://colab.research.google.com/github/MathMachado/DSWP/blob/master/Notebooks/Data%20Aggregation%20and%20Group%20Operations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Aggregation and Group Operations

In [0]:
from numpy.random import randn
import numpy as np
import os
import matplotlib.pyplot as plt
np.random.seed(20111974)

plt.rc('figure', figsize=(10, 6))
from pandas import Series, DataFrame
import pandas as pd
np.set_printoptions(precision=4)

In [0]:
%matplotlib inline

# GroupBy
* Qual a lógica por trás do groupby?

In [0]:
# Dataframe toy
df = DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                'key2' : ['one', 'two', 'one', 'two', 'one'],
                'data1' : np.random.randn(5),
                'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,2.506277,1.049309,a,one
1,1.114404,-0.126074,a,two
2,2.055655,1.062276,b,one
3,0.564824,1.138070,b,two
4,0.298973,1.379660,a,one


In [0]:
df_Grouped= df['data1'].groupby(df['key1'])
df_Grouped

Se quisermos a média:

In [0]:
df_Grouped.mean()

key1
a    1.306551
b    1.310239
Name: data1, dtype: float64

Se quisermos a mediana:

In [0]:
df_Grouped.median()

key1
a    1.114404
b    1.310239
Name: data1, dtype: float64

Se quisermos o mínimo:

In [0]:
df_Grouped.min()

key1
a    0.298973
b    0.564824
Name: data1, dtype: float64

E daí por diante...

Agora, vamos repetir o processo usando as duas chaves:

In [0]:
# Observe o uso da aplicação de mean() ao final da linha
df_Grouped_Mean = df['data1'].groupby([df['key1'], df['key2']]).mean()
# df.groupby(['key1', 'key2'])[['data1']].mean() também funciona
df_Grouped_Mean

key1  key2
a     one     1.402625
      two     1.114404
b     one     2.055655
      two     0.564824
Name: data1, dtype: float64

In [0]:
# Outra forma de ver os resultados:
df_Grouped_Mean.unstack()

key2,one,two
key1,,
a,1.402625,1.114404
b,2.055655,0.564824


### Groupby usando objetos dicionários e séries

In [0]:
df_People = DataFrame(np.random.randn(5, 5),
                   columns=['a', 'b', 'c', 'd', 'e'],
                   index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
df_People.loc['Wes', ['b', 'c']] = np.nan
df_People

,a,b,c,d,e
Joe,-0.502706,-0.153539,-0.808437,0.866650,1.126241
Steve,0.876783,-0.907907,1.370036,0.129628,-0.523001
Wes,0.242031,NaN,NaN,-0.772551,1.510004
Jim,-1.042198,0.489810,0.324786,-1.021235,-1.246583
Travis,1.286583,0.277603,-1.541608,-0.728623,0.823626


In [0]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

In [0]:
by_column = df_People.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,0.058213,0.469996
Steve,1.499665,-0.554126
Wes,-0.772551,1.752035
Jim,-0.696449,-1.798971
Travis,-2.270231,2.387812


In [0]:
map_series = Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [0]:
df_People.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Data Aggregation

In [0]:
df

In [0]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

In [0]:
def peak_to_peak(arr):
    return arr.max() - arr.min()
grouped.agg(peak_to_peak)

In [0]:
grouped.describe()

In [0]:
tips = pd.read_csv('ch08/tips.csv')
# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

### Column-wise and multiple function application

In [0]:
grouped = tips.groupby(['sex', 'smoker'])

In [0]:
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

In [0]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

In [0]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

In [0]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

In [0]:
result['tip_pct']

In [0]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

In [0]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

In [0]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'],
             'size' : 'sum'})

### Returning aggregated data in "unindexed" form

In [0]:
tips.groupby(['sex', 'smoker'], as_index=False).mean()

## Group-wise operations and transformations

In [0]:
df

In [0]:
k1_means = df.groupby('key1').mean().add_prefix('mean_')
k1_means

In [0]:
pd.merge(df, k1_means, left_on='key1', right_index=True)

In [0]:
key = ['one', 'two', 'one', 'two', 'one']
people.groupby(key).mean()

In [0]:
people.groupby(key).transform(np.mean)

In [0]:
def demean(arr):
    return arr - arr.mean()
demeaned = people.groupby(key).transform(demean)
demeaned

In [0]:
demeaned.groupby(key).mean()

### Apply: General split-apply-combine

In [0]:
def top(df, n=5, column='tip_pct'):
    return df.sort_index(by=column)[-n:]
top(tips, n=6)

In [0]:
tips.groupby('smoker').apply(top)

In [0]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

In [0]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

In [0]:
result.unstack('smoker')

f = lambda x: x.describe()
grouped.apply(f)

#### Suppressing the group keys

In [0]:
tips.groupby('smoker', group_keys=False).apply(top)

### Quantile and bucket analysis

In [0]:
frame = DataFrame({'data1': np.random.randn(1000),
                   'data2': np.random.randn(1000)})
factor = pd.cut(frame.data1, 4)
factor[:10]

In [0]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),
            'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(factor)
grouped.apply(get_stats).unstack()

#ADAPT the output is not sorted in the book while this is the case now (swap first two lines)

In [0]:
# Return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

### Example: Filling missing values with group-specific values

In [0]:
s = Series(np.random.randn(6))
s[::2] = np.nan
s

In [0]:
s.fillna(s.mean())

In [0]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']
group_key = ['East'] * 4 + ['West'] * 4
data = Series(np.random.randn(8), index=states)
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

In [0]:
data.groupby(group_key).mean()

In [0]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

In [0]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])

data.groupby(group_key).apply(fill_func)

### Example: Random sampling and permutation

In [0]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (range(1, 11) + [10] * 3) * 4
base_names = ['A'] + range(2, 11) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)

deck = Series(card_val, index=cards)

In [0]:
deck[:13]

In [0]:
def draw(deck, n=5):
    return deck.take(np.random.permutation(len(deck))[:n])
draw(deck)

In [0]:
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

In [0]:
# alternatively
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

### Example: Group weighted average and correlation

In [0]:
df = DataFrame({'category': ['a', 'a', 'a', 'a', 'b', 'b', 'b', 'b'],
                'data': np.random.randn(8),
                'weights': np.random.rand(8)})
df

In [0]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

In [0]:
close_px = pd.read_csv('ch09/stock_px.csv', parse_dates=True, index_col=0)
close_px.info()

In [0]:
close_px[-4:]

In [0]:
rets = close_px.pct_change().dropna()
spx_corr = lambda x: x.corrwith(x['SPX'])
by_year = rets.groupby(lambda x: x.year)
by_year.apply(spx_corr)

In [0]:
# Annual correlation of Apple with Microsoft
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))

### Example: Group-wise linear regression

In [0]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

In [0]:
by_year.apply(regress, 'AAPL', ['SPX'])

## Pivot tables and Cross-tabulation

In [0]:
tips.pivot_table(index=['sex', 'smoker'])

In [0]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],
                 columns='smoker')

In [0]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'],
                 columns='smoker', margins=True)

In [0]:
tips.pivot_table('tip_pct', index=['sex', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

In [0]:
tips.pivot_table('size', index=['time', 'sex', 'smoker'],
                 columns='day', aggfunc='sum', fill_value=0)

### Cross-tabulations: crosstab

In [0]:
from StringIO import StringIO
data = """\
Sample    Gender    Handedness
1    Female    Right-handed
2    Male    Left-handed
3    Female    Right-handed
4    Male    Right-handed
5    Male    Left-handed
6    Male    Right-handed
7    Female    Right-handed
8    Female    Left-handed
9    Male    Right-handed
10    Female    Right-handed"""
data = pd.read_table(StringIO(data), sep='\s+')

In [0]:
data

In [0]:
pd.crosstab(data.Gender, data.Handedness, margins=True)

In [0]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)

## Example: 2012 Federal Election Commission Database

In [0]:
fec = pd.read_csv('ch09/P00000001-ALL.csv')

In [0]:
fec.info()

In [0]:
fec.ix[123456]

In [0]:
unique_cands = fec.cand_nm.unique()
unique_cands

In [0]:
unique_cands[2]

In [0]:
parties = {'Bachmann, Michelle': 'Republican',
           'Cain, Herman': 'Republican',
           'Gingrich, Newt': 'Republican',
           'Huntsman, Jon': 'Republican',
           'Johnson, Gary Earl': 'Republican',
           'McCotter, Thaddeus G': 'Republican',
           'Obama, Barack': 'Democrat',
           'Paul, Ron': 'Republican',
           'Pawlenty, Timothy': 'Republican',
           'Perry, Rick': 'Republican',
           "Roemer, Charles E. 'Buddy' III": 'Republican',
           'Romney, Mitt': 'Republican',
           'Santorum, Rick': 'Republican'}

In [0]:
fec.cand_nm[123456:123461]

In [0]:
fec.cand_nm[123456:123461].map(parties)

In [0]:
# Add it as a column
fec['party'] = fec.cand_nm.map(parties)

In [0]:
fec['party'].value_counts()

In [0]:
(fec.contb_receipt_amt > 0).value_counts()

In [0]:
fec = fec[fec.contb_receipt_amt > 0]

In [0]:
fec_mrbo = fec[fec.cand_nm.isin(['Obama, Barack', 'Romney, Mitt'])]

### Donation statistics by occupation and employer

In [0]:
fec.contbr_occupation.value_counts()[:10]

In [0]:
occ_mapping = {
   'INFORMATION REQUESTED PER BEST EFFORTS' : 'NOT PROVIDED',
   'INFORMATION REQUESTED' : 'NOT PROVIDED',
   'INFORMATION REQUESTED (BEST EFFORTS)' : 'NOT PROVIDED',
   'C.E.O.': 'CEO'
}

# If no mapping provided, return x
f = lambda x: occ_mapping.get(x, x)
fec.contbr_occupation = fec.contbr_occupation.map(f)

In [0]:
emp_mapping = {
   'INFORMATION REQUESTED PER BEST EFFORTS' : 'NOT PROVIDED',
   'INFORMATION REQUESTED' : 'NOT PROVIDED',
   'SELF' : 'SELF-EMPLOYED',
   'SELF EMPLOYED' : 'SELF-EMPLOYED',
}

# If no mapping provided, return x
f = lambda x: emp_mapping.get(x, x)
fec.contbr_employer = fec.contbr_employer.map(f)

In [0]:
by_occupation = fec.pivot_table('contb_receipt_amt',
                                index='contbr_occupation',
                                columns='party', aggfunc='sum')

In [0]:
over_2mm = by_occupation[by_occupation.sum(1) > 2000000]
over_2mm

In [0]:
over_2mm.plot(kind='barh')

In [0]:
def get_top_amounts(group, key, n=5):
    totals = group.groupby(key)['contb_receipt_amt'].sum()

    # Order totals by key in descending order
    return totals.order(ascending=False)[-n:]

In [0]:
grouped = fec_mrbo.groupby('cand_nm')
grouped.apply(get_top_amounts, 'contbr_occupation', n=7)

In [0]:
grouped.apply(get_top_amounts, 'contbr_employer', n=10)

### Bucketing donation amounts

In [0]:
bins = np.array([0, 1, 10, 100, 1000, 10000, 100000, 1000000, 10000000])
labels = pd.cut(fec_mrbo.contb_receipt_amt, bins)
labels

In [0]:
grouped = fec_mrbo.groupby(['cand_nm', labels])
grouped.size().unstack(0)

In [0]:
bucket_sums = grouped.contb_receipt_amt.sum().unstack(0)
bucket_sums

In [0]:
normed_sums = bucket_sums.div(bucket_sums.sum(axis=1), axis=0)
normed_sums

In [0]:
normed_sums[:-2].plot(kind='barh', stacked=True)

### Donation statistics by state

In [0]:
grouped = fec_mrbo.groupby(['cand_nm', 'contbr_st'])
totals = grouped.contb_receipt_amt.sum().unstack(0).fillna(0)
totals = totals[totals.sum(1) > 100000]
totals[:10]

In [0]:
percent = totals.div(totals.sum(1), axis=0)
percent[:10]